In [1]:
import pandas as pd
import numpy as np
import warnings
import sys, os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))
from core.DICT import TYPE2NUM

label_path = '../../data/labels/full_non_transit_radar_labels.csv'
image_path = 'nontransit_track_images/'

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

import warnings
warnings.filterwarnings("ignore", message="Failed to load image Python extension")

Using device: mps


#### CNNBaseline

In [2]:
BATCH_SIZE = 32
NUM_HIDDEN_1 = 32
NUM_HIDDEN_2 = 64
NUM_HIDDEN_3 = 128
IN_CHANNELS = 3

NUM_CLASSES = len(TYPE2NUM)

from dataloader_imgs import get_type_datasets
train_loader, val_loader, test_loader = get_type_datasets(label_path, image_path, batch_size = BATCH_SIZE)

In [3]:
#Check dataloader
for next_batch in train_loader:
    print(next_batch[0].shape)
    print(next_batch[1].shape)
    print(next_batch[1])
    break

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))


torch.Size([32, 3, 224, 224])
torch.Size([32])
tensor([5, 3, 6, 0, 5, 0, 6, 2, 5, 6, 5, 6, 0, 0, 6, 5, 6, 2, 6, 5, 0, 2, 0, 5,
        6, 6, 0, 6, 6, 5, 3, 5])
43
6
6


In [4]:
from models import CNNBaseline
from engine import train_model, save_model

cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_10"

train_config = {
    "model": cnn_baseline_1,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer,
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train and save model
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


Epoch 1/10
Train Loss: 1.6390 | Train Acc: 0.3209
Val   Loss: 1.6139 | Val   Acc: 0.3882


Epoch 2/10
Train Loss: 1.6247 | Train Acc: 0.3385
Val   Loss: 1.6352 | Val   Acc: 0.3882


Epoch 3/10
Train Loss: 1.6153 | Train Acc: 0.3297
Val   Loss: 1.6367 | Val   Acc: 0.3882


Epoch 4/10
Train Loss: 1.5870 | Train Acc: 0.3399
Val   Loss: 1.5833 | Val   Acc: 0.3941


Epoch 5/10
Train Loss: 1.4991 | Train Acc: 0.3955
Val   Loss: 1.4327 | Val   Acc: 0.4588


Epoch 6/10
Train Loss: 1.3955 | Train Acc: 0.4496
Val   Loss: 1.3365 | Val   Acc: 0.4765


KeyboardInterrupt: 

In [5]:
#Retrain for further epochs
cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)

checkpoint_path = f"models/{model_name}.pth"
cnn_baseline_1.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_20"

train_config_resume = {
    "model": cnn_baseline_1,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

# 5. Resume training
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


Epoch 1/10
Train Loss: 1.2608 | Train Acc: 0.5124
Val   Loss: 1.3581 | Val   Acc: 0.4412


Epoch 2/10
Train Loss: 1.1791 | Train Acc: 0.5424
Val   Loss: 1.3505 | Val   Acc: 0.4882


Epoch 3/10
Train Loss: 1.1375 | Train Acc: 0.5621
Val   Loss: 1.3659 | Val   Acc: 0.4588


Epoch 4/10
Train Loss: 1.0781 | Train Acc: 0.5811
Val   Loss: 1.4219 | Val   Acc: 0.4235


Epoch 5/10
Train Loss: 1.0376 | Train Acc: 0.6060
Val   Loss: 1.4253 | Val   Acc: 0.4412


Epoch 6/10
Train Loss: 0.9729 | Train Acc: 0.6221
Val   Loss: 1.4652 | Val   Acc: 0.4412


Epoch 7/10
Train Loss: 0.8995 | Train Acc: 0.6623
Val   Loss: 1.4705 | Val   Acc: 0.4647


Epoch 8/10
Train Loss: 0.8575 | Train Acc: 0.6835
Val   Loss: 1.5046 | Val   Acc: 0.4471


Epoch 9/10
Train Loss: 0.7905 | Train Acc: 0.7032
Val   Loss: 1.5718 | Val   Acc: 0.4765


Epoch 10/10
Train Loss: 0.6831 | Train Acc: 0.7624
Val   Loss: 1.7150 | Val   Acc: 0.4647


### Resnet

In [6]:
from cnn_models import get_resnet50_classifier

resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)
optimizer = optim.Adam(resnet50.parameters(), lr=1e-3)
model_name = "nontransit_resnet50_best_10"

train_config = {
    "model": resnet50,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer, 
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train loop
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)

Epoch 1/10
Train Loss: 1.6453 | Train Acc: 0.3238
Val   Loss: 1.5417 | Val   Acc: 0.3941


Epoch 2/10
Train Loss: 1.4651 | Train Acc: 0.4401
Val   Loss: 1.4376 | Val   Acc: 0.4588


Epoch 3/10
Train Loss: 1.3920 | Train Acc: 0.4773
Val   Loss: 1.3934 | Val   Acc: 0.4412


Epoch 4/10
Train Loss: 1.3304 | Train Acc: 0.5102
Val   Loss: 1.3582 | Val   Acc: 0.4588


Epoch 5/10
Train Loss: 1.2825 | Train Acc: 0.5314
Val   Loss: 1.3382 | Val   Acc: 0.4647


Epoch 6/10
Train Loss: 1.2395 | Train Acc: 0.5461
Val   Loss: 1.3282 | Val   Acc: 0.4882


Epoch 7/10
Train Loss: 1.2262 | Train Acc: 0.5497
Val   Loss: 1.2994 | Val   Acc: 0.5059


Epoch 8/10
Train Loss: 1.1775 | Train Acc: 0.5753
Val   Loss: 1.2962 | Val   Acc: 0.5118


Epoch 9/10
Train Loss: 1.1693 | Train Acc: 0.5702
Val   Loss: 1.2935 | Val   Acc: 0.5176


Epoch 10/10
Train Loss: 1.1438 | Train Acc: 0.5768
Val   Loss: 1.2776 | Val   Acc: 0.5412


In [7]:
resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)

checkpoint_path = "models/nontransit_resnet50_best_10.pth"
resnet50.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(resnet50.parameters(), lr=1e-4)

train_config_resume = {
    "model": resnet50,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

model_name = "nontransit_resnet50_best_20"
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)

Epoch 1/10
Train Loss: 1.1126 | Train Acc: 0.5965
Val   Loss: 1.2686 | Val   Acc: 0.5294


Epoch 2/10
Train Loss: 1.1181 | Train Acc: 0.5892
Val   Loss: 1.2692 | Val   Acc: 0.5059


Epoch 3/10
Train Loss: 1.1027 | Train Acc: 0.6023
Val   Loss: 1.2692 | Val   Acc: 0.5059


Epoch 4/10
Train Loss: 1.1046 | Train Acc: 0.5980
Val   Loss: 1.2721 | Val   Acc: 0.5176


Epoch 5/10
Train Loss: 1.1076 | Train Acc: 0.6001
Val   Loss: 1.2786 | Val   Acc: 0.5118


Epoch 6/10
Train Loss: 1.0981 | Train Acc: 0.6001
Val   Loss: 1.2624 | Val   Acc: 0.5176


Epoch 7/10
Train Loss: 1.0972 | Train Acc: 0.6075
Val   Loss: 1.2679 | Val   Acc: 0.5059


Epoch 8/10
Train Loss: 1.1042 | Train Acc: 0.6075
Val   Loss: 1.2643 | Val   Acc: 0.5118


Epoch 9/10
Train Loss: 1.1143 | Train Acc: 0.5811
Val   Loss: 1.2614 | Val   Acc: 0.4941


Epoch 10/10
Train Loss: 1.0932 | Train Acc: 0.6126
Val   Loss: 1.2626 | Val   Acc: 0.5176
